# 1. Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from rdkit.Chem import MACCSkeys
import warnings
warnings.filterwarnings('ignore')

[16:40:40] Initializing Normalizer


# 2.Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/HDAC3_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  1400 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

1400

# 3.Standardization SDF file for work set

In [5]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  1400 molecules


# 4.Data entry and curation test set

In [6]:
uploaded_file_ts="datasets/HDAC3_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  351 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [7]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [8]:
len(y_ts)

351

# 5.Standardization SDF file for test set

In [9]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  351 molecules


## Calculation MACCS Fingerprints for work set

In [10]:
fp_tr = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ws]

In [11]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [12]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [13]:
savetxt('Models/MACCS/x_tr_MACCS.csv', x_tr, delimiter=',')

In [14]:
x_tr.shape

(1400, 167)

## Calculation  MACCS Fingerprint for test set

In [15]:
fp_ts = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ts]

In [16]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [17]:
x_ts = rdkit_numpy_convert(fp_ts)

In [18]:
x_ts.shape

(351, 167)

In [19]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

 ## GradientBoostingRegressor model building and validation

In [20]:
seed = 42

In [21]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [22]:
param_grid = {'learning_rate': [0.01, 0.04],
                  'subsample'    : [0.9, 0.5],
                  'n_estimators' : [100,1000, 2000],
                  'max_depth'    : [4, 10]
                 }

In [90]:
m = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [91]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=2,
             param_grid={'learning_rate': [0.01, 0.04], 'max_depth': [4, 10],
                         'n_estimators': [100, 1000, 2000],
                         'subsample': [0.9, 0.5]},
             verbose=1)

In [92]:
best_GBR = m.best_estimator_

In [93]:
m.best_params_

{'learning_rate': 0.01,
 'max_depth': 10,
 'n_estimators': 1000,
 'subsample': 0.5}

In [24]:
y_pred_CV_GBR = cross_val_predict(best_GBR, x_tr, y_tr, cv=cv)

In [25]:
y_pred_CV_GBR

array([5.1894373 , 4.97037327, 4.34028259, ..., 8.94338064, 7.33177831,
       9.20135857])

In [50]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_GBR), 2)
Q2_CV

0.65

In [51]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_GBR)), 2)
RMSE_CV

0.72

# 9. Prediction for test set's molecules

In [52]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [53]:
y_pred_GBR = best_GBR.predict(x_ts)

In [54]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.68

In [55]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.7

# save the model to disk

In [42]:
pickle.dump(best_GBR, open('Models/MACCS/HDAC3_GBR_MACCS.pkl', 'wb'))

# load the model from disk

In [23]:
best_GBR = pickle.load(open('Models/MACCS/HDAC3_GBR_MACCS.pkl', 'rb'))

# 10. Y-randomization GBR model

In [101]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_GBR, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  3.8min


True score =  0.65 
Y-randomization =  -0.4 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  7.4min finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [56]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [57]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,2.000000,2.449490,1.414214,2.645751,3.316625,3.000000,2.000000,...,3.162278,2.236068,1.414214,1.000000,1.414214,1.414214,1.000000,0.000000,3.464102,0.000000
2,1.000000,0.000000,3.000000,2.645751,3.872983,3.316625,3.000000,3.605551,3.605551,2.645751,...,3.464102,2.828427,1.414214,1.000000,1.414214,1.414214,1.000000,0.000000,4.000000,0.000000
3,1.000000,0.000000,3.000000,3.000000,3.872983,3.464102,3.741657,3.741657,3.741657,2.828427,...,3.605551,3.464102,1.732051,1.000000,1.414214,1.414214,1.000000,1.000000,4.242640,2.000000
4,1.000000,1.000000,3.316625,3.162278,4.690416,3.464102,3.872983,3.741657,3.741657,2.828427,...,3.741657,3.605551,2.000000,1.000000,1.414214,1.414214,1.000000,1.414214,4.358899,2.236068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,7.874008,7.615773,7.937254,7.874008,8.062258,8.062258,7.874008,7.549834,8.062258,7.937254,...,7.745967,7.745967,8.246211,8.246211,8.366600,8.485281,8.306623,8.366600,8.306623,8.426149
1396,7.874008,7.745967,8.000000,7.874008,8.062258,8.124039,7.874008,7.549834,8.185352,8.000000,...,7.745967,7.745967,8.246211,8.246211,8.366600,8.485281,8.366600,8.426149,8.366600,8.426149
1397,8.000000,7.745967,8.426149,8.000000,8.062258,8.124039,8.000000,7.615773,8.185352,8.062258,...,7.745967,7.810250,8.426149,8.246211,8.426149,8.544003,8.485281,8.544003,8.366600,8.544003
1398,8.062258,7.810250,8.485281,8.062258,8.185352,8.124039,8.000000,7.615773,8.185352,8.062258,...,7.810250,7.874008,8.426149,8.306623,8.426149,8.544003,8.602325,8.660254,8.544003,8.544003


In [58]:
similarity= neighbors_k

In [59]:
Dmean=np.mean(similarity[1,:])

In [60]:
round(Dmean, 2)

1.52

In [61]:
std=np.std(similarity[1,:])

In [62]:
round(std, 2)

1.21

In [63]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

2.12


In [64]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [65]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,341,342,343,344,345,346,347,348,349,350
0,0.000000,3.464102,4.472136,2.645751,2.236068,2.645751,2.828427,1.414214,2.645751,3.162278,...,1.414214,2.828427,2.000000,1.414214,0.000000,0.000000,1.732051,0.000000,2.000000,1.414214
1,0.000000,4.242640,4.690416,2.828427,3.162278,2.828427,2.828427,1.732051,3.000000,3.162278,...,2.000000,3.000000,2.236068,1.732051,0.000000,0.000000,1.732051,1.000000,2.236068,1.732051
2,0.000000,5.000000,4.690416,2.828427,3.316625,3.605551,3.605551,2.000000,3.162278,3.316625,...,3.000000,3.316625,2.449490,1.732051,0.000000,1.000000,2.000000,1.000000,2.236068,2.236068
3,1.000000,5.196152,4.795832,3.000000,3.605551,3.741657,3.872983,2.828427,3.316625,3.316625,...,3.316625,3.464102,3.162278,1.732051,2.000000,1.000000,2.000000,1.000000,2.236068,2.236068
4,1.000000,5.291502,4.795832,3.000000,5.385165,3.741657,4.123106,3.000000,3.464102,3.316625,...,3.316625,3.605551,3.605551,2.000000,2.236068,1.414214,2.236068,1.000000,2.236068,2.449490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,7.937254,7.874008,8.185352,7.874008,8.544003,8.185352,8.306623,8.185352,7.810250,8.306623,...,7.681146,7.874008,7.615773,8.485281,8.426149,8.485281,8.185352,8.185352,8.185352,8.062258
1396,7.937254,7.874008,8.185352,7.937254,8.544003,8.246211,8.306623,8.660254,7.810250,8.366600,...,7.681146,7.874008,7.681146,8.485281,8.426149,8.485281,8.185352,8.185352,8.246211,8.062258
1397,8.062258,7.874008,8.246211,8.000000,8.602325,8.246211,8.306623,8.660254,7.810250,8.366600,...,7.745967,7.937254,7.745967,8.485281,8.544003,8.660254,8.366600,8.185352,8.306623,8.062258
1398,8.124039,7.937254,8.306623,8.000000,8.602325,8.306623,8.306623,8.717798,7.937254,8.366600,...,8.000000,7.937254,7.745967,8.544003,8.544003,8.660254,8.366600,8.246211,8.366600,8.246211


In [66]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[0.    3.464 4.472 2.646 2.236 2.646 2.828 1.414 2.646 3.162 2.449 2.
 3.606 3.606 3.317 1.    1.414 1.414 2.449 2.236 1.    3.742 2.449 0.
 1.414 1.    1.    0.    1.    1.    3.606 1.    2.449 1.414 1.414 1.
 2.236 2.    1.414 2.828 3.    3.464 2.449 3.    2.449 1.732 2.236 2.449
 1.    1.    2.449 1.414 1.732 0.    1.    4.69  1.    0.    0.    0.
 0.    1.    3.    4.69  3.873 0.    3.162 2.    0.    2.646 2.449 1.
 1.732 2.    1.414 0.    1.732 1.    1.732 1.    0.    1.    1.732 0.
 2.828 0.    1.414 1.    0.    3.742 1.    4.123 1.414 2.236 0.    2.646
 1.    0.    0.    1.414 4.899 1.732 3.    0.    1.    0.    2.    0.
 1.414 0.    0.    1.414 1.414 1.    2.646 1.    1.    1.    1.732 1.
 3.162 1.732 3.    2.449 1.    2.828 1.414 1.414 1.    1.    1.414 3.317
 1.    0.    1.414 0.    0.    1.    3.873 1.    3.742 2.236 2.    3.873
 0.    2.    1.    1.414 1.414 2.828 2.449 1.414 0.    0.    0.    2.236
 2.    2.236 1.    1.414 3.873 0.    0.    2.646 0.    1.414 2.    0.
 1.41

In [67]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False False False False False False  True False False False  True
 False False False  True  True  True False False  True False False  True
  True  True  True  True  True  True False  True False  True  True  True
 False  True  True False False False False False False  True False False
  True  True False  True  True  True  True False  True  True  True  True
  True  True False False False  True False  True  True False False  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True False  True False  True False  True False
  True  True  True  True False  True False  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
 False  True False False  True False  True  True  True  True  True False
  True  True  True  True  True  True False  True False False  True False
  True  True  True  True  True False False  True  True  True  True False
  True False  True  True False  True  True False  T

In [68]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7207977207977208


In [69]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   7  11  15  16  17  20  23  24  25  26  27  28  29  31  33  34  35
  37  38  45  48  49  51  52  53  54  56  57  58  59  60  61  65  67  68
  71  72  73  74  75  76  77  78  79  80  81  82  83  85  86  87  88  90
  92  94  96  97  98  99 101 103 104 105 106 107 108 109 110 111 112 113
 115 116 117 118 119 121 124 126 127 128 129 130 132 133 134 135 136 137
 139 142 144 145 146 147 148 151 152 153 154 156 158 159 161 162 164 165
 166 167 168 169 170 171 172 173 174 177 179 180 181 183 184 185 186 187
 188 189 190 191 192 193 196 197 198 199 201 202 203 204 206 207 208 209
 210 211 212 213 214 215 216 217 218 220 221 222 223 225 226 227 228 229
 230 231 233 234 235 237 238 239 240 242 244 245 246 247 248 250 251 252
 253 254 255 256 257 258 259 261 263 264 266 267 268 269 270 271 272 275
 277 280 281 282 283 284 286 287 289 291 294 297 298 299 300 301 303 304
 306 307 308 312 315 316 317 319 320 321 322 323 324 325 326 327 328 329
 331 332 33

In [70]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [71]:
y_pred_GBR_ad=list(y_pred_GBR)

In [72]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [73]:
len(y_pred_GBR_ad)

253

In [74]:
y_ts_ad=list(y_ts)

In [75]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [76]:
len(y_ts_ad)

253

In [77]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.73

In [78]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.62